In [1]:
try:
    from tfx import v1 as tfx
    
    if tfx.__version__ != "1.4.0":
        raise ModuleNotFoundError
except ModuleNotFoundError:
    !pip install tfx==1.4.0

In [2]:
import tensorflow as tf

from tfx.orchestration.experimental.interactive.interactive_context import InteractiveContext

%load_ext tfx.orchestration.experimental.interactive.notebook_extensions.skip

print('TFX version: {}'.format(tfx.__version__))
print('TensorFlow version: {}'.format(tf.__version__))

TFX version: 1.4.0
TensorFlow version: 2.6.2


In [3]:
import os
from absl import logging

logging.set_verbosity(logging.INFO)

PIPELINE_NAME = "iris"
PIPELINE_ROOT = os.path.join('pipelines', PIPELINE_NAME)
METADATA_PATH = os.path.join('metadata', PIPELINE_NAME, 'metadata.db')
SERVING_MODEL_DIR = os.path.join('serving_model', PIPELINE_NAME)

DATA_ROOT = os.path.join("data")
os.makedirs(DATA_ROOT, exist_ok=True)

# Retrieve data

In [4]:
import urllib.request

_data_url = 'https://datahub.io/machine-learning/iris/r/iris.csv'
_data_filepath = os.path.join(DATA_ROOT, "data.csv")

urllib.request.urlretrieve(_data_url, _data_filepath)

!head {_data_filepath}

# Create TFX pipeline and run it

In [5]:
_module_file = 'iris_utils.py'

In [6]:
def _create_pipeline(pipeline_name: str, pipeline_root: str, data_root: str,
                     module_file: str, serving_model_dir: str,
                     metadata_path: str) -> tfx.dsl.Pipeline:
    # Brings data into the pipeline or otherwise joins/converts training data.
    example_gen = tfx.components.CsvExampleGen(
        input_base=data_root)

    # Computes statistics over data for visualization and example validation.
    statistics_gen = tfx.components.StatisticsGen(
        examples=example_gen.outputs['examples'])

    # Generate a schema based on your data statistics.
    # A schema defines the expected bounds, types, and properties of the features in your dataset.
    schema_gen = tfx.components.SchemaGen(
        statistics=statistics_gen.outputs['statistics'], infer_feature_shape=True)

    # Performs anomaly detection based on statistics and data schema.
    example_validator = tfx.components.ExampleValidator(
        statistics=statistics_gen.outputs['statistics'],
        schema=schema_gen.outputs['schema'])

    # Transforms input data using preprocessing_fn in the 'module_file'.
    transform = tfx.components.Transform(
        examples=example_gen.outputs['examples'],
        schema=schema_gen.outputs['schema'],
        materialize=False,
        module_file=module_file)

    # Uses user-provided Python function that trains a model.
    trainer = tfx.components.Trainer(
        module_file=module_file,
        examples=example_gen.outputs['examples'],
        schema=schema_gen.outputs['schema'],
        transform_graph=transform.outputs['transform_graph'],
        train_args=tfx.proto.TrainArgs(num_steps=2000),
        eval_args=tfx.proto.EvalArgs(num_steps=5))

    # Pushes the model to a filesystem destination.
    pusher = tfx.components.Pusher(
        model=trainer.outputs['model'],
        push_destination=tfx.proto.PushDestination(filesystem=tfx.proto.PushDestination
                                                   .Filesystem(base_directory=serving_model_dir))
    )

    components = [
        example_gen,
        statistics_gen,
        schema_gen,
        example_validator,
        transform,
        trainer,
        pusher,
    ]

    return tfx.dsl.Pipeline(
        pipeline_name=pipeline_name,
        pipeline_root=pipeline_root,
        metadata_connection_config=tfx.orchestration.metadata.sqlite_metadata_connection_config(metadata_path),
        components=components)

In [7]:
pipeline = _create_pipeline(
    pipeline_name=PIPELINE_NAME,
    pipeline_root=PIPELINE_ROOT,
    data_root=DATA_ROOT,
    module_file=_module_file,
    serving_model_dir=SERVING_MODEL_DIR,
    metadata_path=METADATA_PATH
)

tfx.orchestration.LocalDagRunner().run(pipeline)

INFO:absl:Excluding no splits because exclude_splits is not set.
INFO:absl:Excluding no splits because exclude_splits is not set.
INFO:absl:Excluding no splits because exclude_splits is not set.
INFO:absl:Generating ephemeral wheel package for '/home/ekrem/Workplace/Python/tfx_seminar/iris_utils.py' (including modules: ['iris_utils']).
INFO:absl:User module package has hash fingerprint version 60fea989e6fd0be846ae09aa972872d3dfc1a73e26eb8bce6d01368ffc6b3d48.
INFO:absl:Executing: ['/home/ekrem/.pyenv/versions/3.8.7/envs/tfx/bin/python3.8', '/tmp/tmpj8jnz0yo/_tfx_generated_setup.py', 'bdist_wheel', '--bdist-dir', '/tmp/tmp0pa53bsa', '--dist-dir', '/tmp/tmp7lfzkrbm']
/home/ekrem/.pyenv/versions/3.8.7/envs/tfx/lib/python3.8/site-packages/setuptools/command/install.py:34: SetuptoolsDeprecationWarning: setup.py install is deprecated. Use build and pip and other standards-based tools.
  warnings.warn(
INFO:absl:Successfully built user code wheel distribution at 'pipelines/iris/_wheels/tfx_use

running bdist_wheel
running build
running build_py
creating build
creating build/lib
copying iris_utils.py -> build/lib
installing to /tmp/tmp0pa53bsa
running install
running install_lib
copying build/lib/iris_utils.py -> /tmp/tmp0pa53bsa
running install_egg_info
running egg_info
creating tfx_user_code_Transform.egg-info
writing tfx_user_code_Transform.egg-info/PKG-INFO
writing dependency_links to tfx_user_code_Transform.egg-info/dependency_links.txt
writing top-level names to tfx_user_code_Transform.egg-info/top_level.txt
writing manifest file 'tfx_user_code_Transform.egg-info/SOURCES.txt'
reading manifest file 'tfx_user_code_Transform.egg-info/SOURCES.txt'
writing manifest file 'tfx_user_code_Transform.egg-info/SOURCES.txt'
Copying tfx_user_code_Transform.egg-info to /tmp/tmp0pa53bsa/tfx_user_code_Transform-0.0+60fea989e6fd0be846ae09aa972872d3dfc1a73e26eb8bce6d01368ffc6b3d48-py3.8.egg-info
running install_scripts
creating /tmp/tmp0pa53bsa/tfx_user_code_Transform-0.0+60fea989e6fd0be84

/home/ekrem/.pyenv/versions/3.8.7/envs/tfx/lib/python3.8/site-packages/setuptools/command/install.py:34: SetuptoolsDeprecationWarning: setup.py install is deprecated. Use build and pip and other standards-based tools.
  warnings.warn(
INFO:absl:Successfully built user code wheel distribution at 'pipelines/iris/_wheels/tfx_user_code_Trainer-0.0+60fea989e6fd0be846ae09aa972872d3dfc1a73e26eb8bce6d01368ffc6b3d48-py3-none-any.whl'; target user module is 'iris_utils'.
INFO:absl:Full user module path is 'iris_utils@pipelines/iris/_wheels/tfx_user_code_Trainer-0.0+60fea989e6fd0be846ae09aa972872d3dfc1a73e26eb8bce6d01368ffc6b3d48-py3-none-any.whl'
INFO:absl:Using deployment config:
 executor_specs {
  key: "CsvExampleGen"
  value {
    beam_executable_spec {
      python_executor_spec {
        class_path: "tfx.components.example_gen.csv_example_gen.executor.Executor"
      }
    }
  }
}
executor_specs {
  key: "ExampleValidator"
  value {
    python_class_executable_spec {
      class_path: "tfx

INFO:absl:Generating examples.


running bdist_wheel
running build
running build_py
creating build
creating build/lib
copying iris_utils.py -> build/lib
installing to /tmp/tmpfokasgd3
running install
running install_lib
copying build/lib/iris_utils.py -> /tmp/tmpfokasgd3
running install_egg_info
running egg_info
creating tfx_user_code_Trainer.egg-info
writing tfx_user_code_Trainer.egg-info/PKG-INFO
writing dependency_links to tfx_user_code_Trainer.egg-info/dependency_links.txt
writing top-level names to tfx_user_code_Trainer.egg-info/top_level.txt
writing manifest file 'tfx_user_code_Trainer.egg-info/SOURCES.txt'
reading manifest file 'tfx_user_code_Trainer.egg-info/SOURCES.txt'
writing manifest file 'tfx_user_code_Trainer.egg-info/SOURCES.txt'
Copying tfx_user_code_Trainer.egg-info to /tmp/tmpfokasgd3/tfx_user_code_Trainer-0.0+60fea989e6fd0be846ae09aa972872d3dfc1a73e26eb8bce6d01368ffc6b3d48-py3.8.egg-info
running install_scripts
creating /tmp/tmpfokasgd3/tfx_user_code_Trainer-0.0+60fea989e6fd0be846ae09aa972872d3dfc1a

...

2021-11-17 12:17:40.920138: I tensorflow/core/profiler/lib/profiler_session.cc:131] Profiler session initializing.
2021-11-17 12:17:40.920167: I tensorflow/core/profiler/lib/profiler_session.cc:146] Profiler session started.


  29/2000 [..............................] - ETA: 30s - loss: 0.6724 - sparse_categorical_accuracy: 0.7655

2021-11-17 12:17:41.182541: I tensorflow/core/profiler/lib/profiler_session.cc:66] Profiler session collecting data.
2021-11-17 12:17:41.183660: I tensorflow/core/profiler/internal/gpu/cupti_tracer.cc:1748] CUPTI activity buffer flushed
2021-11-17 12:17:41.225277: I tensorflow/core/profiler/internal/gpu/cupti_collector.cc:673]  GpuTracer has collected 86 callback api events and 83 activity events. 
2021-11-17 12:17:41.227425: I tensorflow/core/profiler/lib/profiler_session.cc:164] Profiler session tear down.
2021-11-17 12:17:41.230804: I tensorflow/core/profiler/rpc/client/save_profile.cc:136] Creating directory: pipelines/iris/Trainer/model_run/32/train/plugins/profile/2021_11_17_12_17_41

2021-11-17 12:17:41.232816: I tensorflow/core/profiler/rpc/client/save_profile.cc:142] Dumped gzipped tool data for trace.json.gz to pipelines/iris/Trainer/model_run/32/train/plugins/profile/2021_11_17_12_17_41/EkE-Legion5.trace.json.gz
2021-11-17 12:17:41.237359: I tensorflow/core/profiler/rpc/clie

2000/2000 [==============================] - 8s 4ms/step - loss: 0.0214 - sparse_categorical_accuracy: 0.9923 - val_loss: 1.1805 - val_sparse_categorical_accuracy: 0.9200
INFO:tensorflow:Assets written to: pipelines/iris/Trainer/model/32/Format-Serving/assets


INFO:tensorflow:Assets written to: pipelines/iris/Trainer/model/32/Format-Serving/assets
INFO:absl:Training complete. Model written to pipelines/iris/Trainer/model/32/Format-Serving. ModelRun written to pipelines/iris/Trainer/model_run/32
INFO:absl:Cleaning up stateless execution info.
INFO:absl:Execution 32 succeeded.
INFO:absl:Cleaning up stateful execution info.
INFO:absl:Publishing output artifacts defaultdict(<class 'list'>, {'model': [Artifact(artifact: uri: "pipelines/iris/Trainer/model/32"
custom_properties {
  key: "name"
  value {
    string_value: "iris:2021-11-17T12:17:22.726331:Trainer:model:0"
  }
}
custom_properties {
  key: "tfx_version"
  value {
    string_value: "1.4.0"
  }
}
, artifact_type: name: "Model"
)], 'model_run': [Artifact(artifact: uri: "pipelines/iris/Trainer/model_run/32"
custom_properties {
  key: "name"
  value {
    string_value: "iris:2021-11-17T12:17:22.726331:Trainer:model_run:0"
  }
}
custom_properties {
  key: "tfx_version"
  value {
    string_v

# Visualize pipeline

In [8]:
from tfx.orchestration.portable.mlmd import execution_lib
from tfx.orchestration.metadata import Metadata
from tfx.orchestration.experimental.interactive import visualizations
from ml_metadata.proto import metadata_store_pb2


def get_latest_artifact(component_id):
    metadata_connection_config = tfx.orchestration.metadata.sqlite_metadata_connection_config(METADATA_PATH)

    with Metadata(metadata_connection_config) as metadata_handler:
        context = metadata_handler.store.get_context_by_type_and_name('node', f'{PIPELINE_NAME}.{component_id}')
        executions = metadata_handler.store.get_executions_by_context(context.id)
        latest_execution = max(executions, key=lambda e: e.last_update_time_since_epoch)

        return execution_lib.get_artifacts_dict(metadata_handler, latest_execution.id, [metadata_store_pb2.Event.OUTPUT])


def visualize(artifact):
    visualization = visualizations.get_registry().get_visualization(artifact.type_name)
    visualization.display(artifact)


from tfx.orchestration.experimental.interactive import standard_visualizations

standard_visualizations.register_standard_visualizations()

## StatisticsGen

In [9]:
statistic_artifact = get_latest_artifact("StatisticsGen")['statistics'][0]

visualize(statistic_artifact)

INFO:absl:MetadataStore with DB connection initialized


## SchemaGen

In [10]:
schema_artifact = get_latest_artifact("SchemaGen")['schema'][0]

visualize(schema_artifact)

INFO:absl:MetadataStore with DB connection initialized


,Type,Presence,Valency,Domain
Feature name,,,,
'class',STRING,required,,'class'
'petallength',FLOAT,required,,-
'petalwidth',FLOAT,required,,-
'sepallength',FLOAT,required,,-
'sepalwidth',FLOAT,required,,-


,Values
Domain,
'class',"'Iris-setosa', 'Iris-versicolor', 'Iris-virginica'"


## ExampleValidator

In [11]:
example_validator_articaft = get_latest_artifact("ExampleValidator")['anomalies'][0]

visualize(example_validator_articaft)

INFO:absl:MetadataStore with DB connection initialized


## Transformer

In [12]:
# TODO

## Trainer

In [13]:
model_run_artifact_dir = get_latest_artifact("Trainer")['model_run'][0].uri

%load_ext tensorboard
%tensorboard --logdir {model_run_artifact_dir}

INFO:absl:MetadataStore with DB connection initialized


# Prediction

Serve model by running this in your shell:

```
$ tensorflow_model_server --rest_api_port=8501 --model_name=iris --model_base_path=.../tfx_seminar/serving_model/iris
```

In [37]:
import json
import requests

URL = 'http://localhost:8501/v1/models/' + PIPELINE_NAME + ':predict'
HEADERS = {"content-type": "application/json"}

def prediction_request(sepallength, sepalwidth, petallength, petalwidth):
    request_data = json.dumps({"instances": [{
        "petallength": [petallength],
        "petalwidth": [petalwidth],
        "sepallength": [sepallength],
        "sepalwidth": [sepalwidth]
        }]
    })
    
    response = requests.post(data=request_data, url=URL, headers=HEADERS)
    
    return json.loads(response.text)

prediction_request(4.4,3.2,1.3,0.2)

{'predictions': [[-19.3413448, 15.2059145, -13.3737993]]}